名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [1]:
import re

In [2]:
# 区切り文字
separator = re.compile('\t|,')

# 係り受け
dependancy = re.compile(r'''(?:\*\s\d+\s) # キャプチャ対象外
                            (-?\d+)       # 数字(係り先)
                          ''', re.VERBOSE)

In [3]:
class Morph:
    def __init__(self, line):
        
        #タブとカンマで分割
        cols = separator.split(line)
        
        self.surface = cols[0] # 表層形(surface)
        self.base = cols[7]    # 基本形(base)
        self.pos = cols[1]     # 品詞(pos)
        self.pos1 = cols[2]    # 品詞細分類1(pos1)

In [4]:
class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs
        self.srcs = []   # 係り元文節インデックス番号のリスト
        self.dst  = dst  # 係り先文節インデックス番号
        
        self.verb = False
        self.noun = False
        self.phrase = ''
        
        for morph in morphs:
            
            # 記号以外の場合文節作成
            if morph.pos != '記号':
                self.phrase += morph.surface
            if morph.pos == '動詞':
                self.verb = True
            if morph.pos == '名詞':
                self.noun = True

In [5]:
# 係り元を代入し、Chunkリストを文のリストを追加
def append_sentence(chunks, sentences):
    
    # 係り元を代入
    for i, chunk in enumerate(chunks):
        if chunk.dst != -1:
            chunks[chunk.dst].srcs.append(i)
    sentences.append(chunks)
    return sentences, []

In [6]:
%time

morphs = []
chunks = []
sentences = []

with open('./neko.txt.cabocha') as f:
    
    for line in f:
        dependancies = dependancy.match(line)
        
        # EOSまたは係り受け解析結果でない場合
        if not (line == 'EOS\n' or dependancies):
            morphs.append(Morph(line))
            
        # EOSまたは係り受け解析結果で、形態素解析結果がある場合
        elif len(morphs) > 0:
            chunks.append(Chunk(morphs, dst))
            morphs = []
       
        # 係り受け結果の場合
        if dependancies:
            dst = int(dependancies.group(1))
        
        # EOSで係り受け結果がある場合
        if line == 'EOS\n' and len(chunks) > 0:
            sentences, chunks = append_sentence(chunks, sentences)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.34 µs


In [7]:
for i, sentence in enumerate(sentences):
    for chunk in sentence:
        if chunk.dst != -1 and \
           chunk.noun and \
           sentence[chunk.dst].verb:
            print('{}\t{}'.format(chunk.phrase, sentence[chunk.dst].phrase))
    
    # 多いので制限
    if i > 50:
        break

どこで	生れた
かとんと	つかぬ
見当が	つかぬ
した所で	泣いて
いた事だけは	記憶している
吾輩は	見た
ここで	始めて
ものを	見た
あとで	聞くと
我々を	捕えて
掌に	載せられて
スーと	持ち上げられた
時	フワフワした
感じが	あったばかりである
上で	落ちついて
顔を	見たのが
ものの	見始であろう
ものだと	思った
感じが	残っている
今でも	残っている
第一毛をもって	装飾されべきはずの
顔が	つるつるして
その後	逢ったが
猫にも	逢ったが
一度も	出会わし
真中が	突起している
中から	吹く
煙を	吹く
咽せぽくて	弱った
人間の	飲む
事は	知った
頃	知った
裏で	坐って
心持に	坐って
速力で	運転し始めた
書生が	動くのか
動くのか	動くのか
自分だけが	動くのか
動くのか	分らないが
眼が	廻る
胸が	悪くなる
音が	して
眼から	出た
火が	出た
それまでは	記憶しているが
記憶しているが	分らない
あとは	分らない
事やら	分らない
気が	付いて
書生は	いない
たくさん	おった
兄弟が	見えぬ
一疋も	見えぬ
母親さえ	隠してしまった
姿を	隠してしまった
所とは	違って
眼を	明いていられぬくらいだ
吾輩は	棄てられたのである
上から	棄てられたのである
急に	棄てられたのである
中へ	棄てられたのである
思いで	這い出すと
笹原を	這い出すと
向うに	ある
池が	ある
吾輩は	見た
前に	坐って
分別も	出ない
書生がまた	来てくれるかと
迎に	来てくれるかと
ニャーと	やって
誰も	来ない
池の上を	渡って
風が	渡って
日が	かかる
暮れ	かかる
腹が	減って来た
非常に	減って来た
食物の	ある
所まで	ある
決心を	して
池を	廻り始めた
左りに	廻り始めた
そこを	我慢して
我慢して	這って行くと
無理やりに	這って行くと
事で	出た
所へ	出た
ここへ	這入ったら
竹垣の	崩れた
穴から	もぐり込んだ
邸内に	もぐり込んだ
もので	餓死したかも知れんのである
竹垣が	破れていなかったなら
吾輩は	餓死したかも知れんのである
路傍に	餓死したかも知れんのである
蔭とは	云った
穴は	なっている
今日に	至るまで
吾輩が	訪問する
三毛を	訪問する
通路に	なっている
邸へは	忍び込んだものの
うちに	暗くなる
腹は